# Phase 3 Weighted Bagging

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

import os
import re
import csv
import codecs
import gensim
import itertools
import numpy as np
import pandas as pd
import operator
import sys

from nltk import ngrams
from collections import Counter
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from iwillwin.data_utils.feature_engineering import FeatureCreator

from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize

import seaborn as sns
%matplotlib inline

import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
import operator
from os import listdir
from os.path import isfile, join

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.config import dataset_config
from keras.utils import to_categorical

C:\Users\zake7\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\zake7\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
NB_WORDS, MAX_SEQUENCE_LENGTH = 50000, 50
data_transformer = DataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains_nns, tests_nns, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zake7\AppData\Local\Temp\jieba.cache
Loading model cost 0.465 seconds.
Prefix dict has been built succesfully.


[DataHelper] Apply normalization on value-type columns


C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 50) (320552, 50)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 83265


In [3]:
trains_meta = trains_nns[2]
tests_meta = tests_nns[2]

In [4]:
train_df = pd.read_csv('../data/dataset/train.csv')
test_df = pd.read_csv('../data/dataset/test.csv')

In [5]:
rumor_words = ['辟谣', '谣言', '勿传', '假的']

def is_rumor(text):
    if type(text) != str:
        print(text, type(text))
        return 0
    for rumor_word in rumor_words:
        if rumor_word in text:
            return 1
    return 0

def has_split_symbol(text):
    if type(text) != str:
        return 0
    if '|' in text:
        return 1
    return 0

for df in [train_df, test_df]:
    df['has_|'] = df['title2_zh'].apply(has_split_symbol)
    df['has_rumor_words'] = df['title2_zh'].apply(is_rumor)

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [6]:
train_has_rumor = train_df.has_rumor_words.values
test_has_rumor = test_df.has_rumor_words.values

trick_trains_features = np.concatenate((trains_nns[2], train_has_rumor.reshape((-1, 1))), axis=1)
trick_tests_features = np.concatenate((tests_nns[2], test_has_rumor.reshape((-1, 1))), axis=1)

In [7]:
oof_file_names = sorted([f for f in listdir('../data/ensemble/oofs/') if isfile(join('../data/ensemble/oofs/', f)) and f != '.gitkeep'])
preds_file_names = [name.replace('-Train', '') for name in oof_file_names]

oofs = []
preds = []
for name in oof_file_names:
    oofs.append(pd.read_csv('../data/ensemble/oofs/' + name))
for name in preds_file_names:
    preds.append(pd.read_csv('../data/ensemble/preds/' + name))    

In [8]:
for i, name in enumerate(oof_file_names):
    print(i, name)
    
trains = pd.DataFrame()
tests = pd.DataFrame()

for i in range(len(oof_file_names)):
    for label_type in ['agreed', 'disagreed', 'unrelated']:
        trains['oofs_{}_{}'.format(i, label_type)] = oofs[i][label_type].values
        tests['oofs_pred{}_{}'.format(i, label_type)] = preds[i][label_type].values

0 AddNN-Ensemble-Train-L0.861201-NB50000.csv
1 LightGBM-Ensemble-Train-L0.860117-NB50000.csv
2 LightGBMWordLevel-Ensemble-Train-L0.859096-NB50000.csv
3 Logistic-Ensemble-Train-L0.860191-NB50000.csv
4 Ridge-Ensemble-Train-L0.859993-NB50000.csv


In [9]:
unrelated = pd.DataFrame()
agreeds = pd.DataFrame()
disagreeds = pd.DataFrame()

#check_oofs = True
check_oofs = False


if check_oofs:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = oofs[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = oofs[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = oofs[i]['disagreed'].values
else:
    for i, oof in enumerate(oofs):
        agreeds['oofs_agreed_{}'.format(i)] = preds[i]['agreed'].values
        unrelated['oofs_unrelated_{}'.format(i)] = preds[i]['unrelated'].values
        disagreeds['oofs_disagreeds_{}'.format(i)] = preds[i]['disagreed'].values  

In [10]:
agreeds.corr()

,oofs_agreed_0,oofs_agreed_1,oofs_agreed_2,oofs_agreed_3,oofs_agreed_4
oofs_agreed_0,1.000000,0.999462,0.998657,0.995063,0.998853
oofs_agreed_1,0.999462,1.000000,0.999139,0.994128,0.998696
oofs_agreed_2,0.998657,0.999139,1.000000,0.992774,0.997810
oofs_agreed_3,0.995063,0.994128,0.992774,1.000000,0.995594
oofs_agreed_4,0.998853,0.998696,0.997810,0.995594,1.000000


In [11]:
disagreeds.corr()

,oofs_disagreeds_0,oofs_disagreeds_1,oofs_disagreeds_2,oofs_disagreeds_3,oofs_disagreeds_4
oofs_disagreeds_0,1.000000,0.995894,0.995529,0.954959,0.934220
oofs_disagreeds_1,0.995894,1.000000,0.997784,0.948563,0.926715
oofs_disagreeds_2,0.995529,0.997784,1.000000,0.948111,0.926025
oofs_disagreeds_3,0.954959,0.948563,0.948111,1.000000,0.925313
oofs_disagreeds_4,0.934220,0.926715,0.926025,0.925313,1.000000


In [13]:
unrelated.corr()

,oofs_unrelated_0,oofs_unrelated_1,oofs_unrelated_2,oofs_unrelated_3,oofs_unrelated_4
oofs_unrelated_0,1.000000,0.999235,0.998382,0.992162,0.998741
oofs_unrelated_1,0.999235,1.000000,0.998972,0.990868,0.998143
oofs_unrelated_2,0.998382,0.998972,1.000000,0.989487,0.997331
oofs_unrelated_3,0.992162,0.990868,0.989487,1.000000,0.990611
oofs_unrelated_4,0.998741,0.998143,0.997331,0.990611,1.000000


# Prepare Different Inputs

In [14]:
# Only use oofs
ensemble_trains = trains.values
ensemble_tests = tests.values

In [45]:
# use oofs and meta-features
#ensemble_trains = np.concatenate((trains.values, trick_trains_features), axis=1)
#ensemble_tests = np.concatenate((tests.values, trick_tests_features), axis=1)

# Ensemble With NN

In [15]:
########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader
from iwillwin.model.sim_zoos import *
import tensorflow as tf
from keras.layers import Dense, Input, MaxPooling1D, CuDNNLSTM, Embedding, Add, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from iwillwin.config import dataset_config, model_config
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Lambda, Dense, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras.legacy.layers import Highway
from keras.layers import TimeDistributed
from keras.layers.normalization import BatchNormalization
import keras.backend as K

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import log_loss

In [16]:
from keras import regularizers

def weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    
    y_pred = K.cast(y_pred > 0.5, 'int32') # Hard
    y_true = K.cast(y_true, 'int32')
    
    res = K.cast(K.equal(y_pred, y_true), 'float32') * weight_mask / K.sum(weight_mask)
    res = K.sum(res)
    return res

def get_dense_add_net(feature_nums):
    features_inputs = Input(shape=(feature_nums,), name='mata-features', dtype="float32")
    features = features_inputs
    
    depth = 5
    for i in range(depth):
        new_features = Dense(24, activation='relu')(features)
        new_features = Dropout(0.1)(new_features)
        features = Concatenate()([features, new_features])

    h = Highway(activation='relu')(features)
    out_ = Dense(3, activation='softmax')(h)
    
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [17]:
def get_linear_net(feature_nums):
    features_inputs = Input(shape=(feature_nums,), name='mata-features', dtype="float32")    
    out_ = Dense(3, activation='softmax')(features_inputs)
    model = Model(inputs=[features_inputs], outputs=out_)
    model.compile(optimizer=Adam(lr=1e-3, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [37]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import importlib

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint

from iwillwin.config import model_config

class ModelTrainer(object):

    def __init__(self, model_stamp, epoch_num, learning_rate=1e-3,
                 shuffle_inputs=False, verbose_round=40, early_stopping_round=8):
        self.models = []
        self.model_stamp = model_stamp
        self.val_loss = -1
        self.auc = -1
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.eps = 1e-10
        self.verbose_round = verbose_round
        self.early_stopping_round = early_stopping_round
        self.shuffle_inputs = shuffle_inputs
        self.class_weight = [0.93, 1.21]

    def train_folds(self, features, y, fold_count, batch_size, get_model_func, augments=None, skip_fold=0, patience=10, scale_sample_weight=False,
                    class_weight=None, self_aware=False, swap_input=False):
        weight_val=scale_sample_weight
        class_weight=None
        fold_size = len(features) // fold_count
        models = []
        fold_predictions = []
        score = 0

        for fold_id in range(0, fold_count):
            fold_start = fold_size * fold_id
            fold_end = fold_start + fold_size

            if fold_id == fold_count - 1:
                fold_end = len(features)

            train_features = np.concatenate([features[:fold_start], features[fold_end:]])
            train_y = np.concatenate([y[:fold_start], y[fold_end:]])
            
            val_features = features[fold_start:fold_end]
            val_y = y[fold_start:fold_end]
            fold_pos = (np.sum(train_y) / len(train_features))

            train_data = {
                "mata-features": train_features,
            }

            val_data = {
                "mata-features": val_features,
            }

            model, bst_val_score, fold_prediction = self._train_model_by_logloss(
                get_model_func(), batch_size, train_data, train_y, val_data, val_y, fold_id, patience, class_weight, weight_val=None)
    
            score += bst_val_score
            models.append(model)
            fold_predictions.append(fold_prediction)

        self.models = models
        self.val_loss = score / fold_count
        return models, self.val_loss, fold_predictions

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience):
        # return a list which holds [models, val_loss, auc, prediction]
        raise NotImplementedError

class KerasModelTrainer(ModelTrainer):

    def __init__(self, *args, **kwargs):
        super(KerasModelTrainer, self).__init__(*args, **kwargs)
        pass

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience, class_weight, weight_val):
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        bst_model_path = self.model_stamp + str(fold_id) + '.h5'
        val_data = (val_x, val_y, weight_val) if weight_val is not None else (val_x, val_y)
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
        hist = model.fit(train_x, train_y,
                         validation_data=val_data,
                         epochs=self.epoch_num, batch_size=batch_size, shuffle=True,
                         verbose=2,
                         class_weight={0: 1/16, 1: 1/15, 2: 1/5},
                         callbacks=[early_stopping, model_checkpoint],)
        bst_val_score = max(hist.history['val_weighted_accuracy'])
        model.load_weights(bst_model_path)
        predictions = model.predict(val_x)

        return model, bst_val_score, predictions

In [38]:
def _agent_get_model():
    return get_dense_add_net(ensemble_trains.shape[1])

In [39]:
def np_weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    weight_mask = np.max(weight_mask, axis=-1)
    norms = np.sum(weight_mask)
    
    y_true = np.argmax(y_true, axis=-1)
    y_pred = np.argmax(y_pred, axis=-1)
    
    res = ((y_true == y_pred) * weight_mask).sum() / norms
    return res

In [40]:
def weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    
    y_pred = K.cast(y_pred > 0.5, 'int32') # Hard
    y_true = K.cast(y_true, 'int32')
    
    res = K.cast(K.equal(y_pred, y_true), 'float32') * weight_mask / K.sum(weight_mask)
    res = K.sum(res)
    return res

In [41]:
trainer = KerasModelTrainer(model_stamp="Ensemble-DenseNet", epoch_num=500)
models, score, folds_preds = trainer.train_folds(features=ensemble_trains, y=to_categorical(labels), augments=None, fold_count=10,
    batch_size=1024, 
    scale_sample_weight=None, class_weight=None,
    get_model_func=_agent_get_model, 
    patience=20)

C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:198: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 15)           0                                            
__________________________________________________________________________________________________
dense_73 (Dense)                (None, 24)           384         mata-features[0][0]              
__________________________________________________________________________________________________
dropout_61 (Dropout)            (None, 24)           0           dense_73[0][0]                   
__________________________________________________________________________________________________
concatenate_61 (Concatenate)    (None, 39)           0           mata-features[0][0]              
                                                                 dropout_61[0][0]                 
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mata-features (InputLayer)      (None, 15)           0                                            
__________________________________________________________________________________________________
dense_85 (Dense)                (None, 24)           384         mata-features[0][0]              
__________________________________________________________________________________________________
dropout_71 (Dropout)            (None, 24)           0           dense_85[0][0]                   
__________________________________________________________________________________________________
concatenate_71 (Concatenate)    (None, 39)           0           mata-features[0][0]              
                                                                 dropout_71[0][0]                 
__________

dense_100 (Dense)               (None, 24)           2112        concatenate_83[0][0]             
__________________________________________________________________________________________________
dropout_84 (Dropout)            (None, 24)           0           dense_100[0][0]                  
__________________________________________________________________________________________________
concatenate_84 (Concatenate)    (None, 111)          0           concatenate_83[0][0]             
                                                                 dropout_84[0][0]                 
__________________________________________________________________________________________________
dense_101 (Dense)               (None, 24)           2688        concatenate_84[0][0]             
__________________________________________________________________________________________________
dropout_85 (Dropout)            (None, 24)           0           dense_101[0][0]                  
__________

highway_19 (Highway)            (None, 135)          36720       concatenate_95[0][0]             
__________________________________________________________________________________________________
dense_114 (Dense)               (None, 3)            408         highway_19[0][0]                 
Total params: 44,808
Trainable params: 44,808
Non-trainable params: 0
__________________________________________________________________________________________________
Train on 288497 samples, validate on 32055 samples
Epoch 1/500
 - 3s - loss: 0.0220 - acc: 0.8775 - weighted_accuracy: 0.8534 - val_loss: 0.3005 - val_acc: 0.8607 - val_weighted_accuracy: 0.8556
Epoch 2/500
 - 1s - loss: 0.0199 - acc: 0.8806 - weighted_accuracy: 0.8706 - val_loss: 0.2995 - val_acc: 0.8645 - val_weighted_accuracy: 0.8541
Epoch 3/500
 - 1s - loss: 0.0197 - acc: 0.8805 - weighted_accuracy: 0.8706 - val_loss: 0.3012 - val_acc: 0.8632 - val_weighted_accuracy: 0.8552
Epoch 4/500
 - 1s - loss: 0.0197 - acc: 0.8804 - we

__________________________________________________________________________________________________
dropout_102 (Dropout)           (None, 24)           0           dense_122[0][0]                  
__________________________________________________________________________________________________
concatenate_102 (Concatenate)   (None, 63)           0           concatenate_101[0][0]            
                                                                 dropout_102[0][0]                
__________________________________________________________________________________________________
dense_123 (Dense)               (None, 24)           1536        concatenate_102[0][0]            
__________________________________________________________________________________________________
dropout_103 (Dropout)           (None, 24)           0           dense_123[0][0]                  
__________________________________________________________________________________________________
concatenat

In [48]:
score

0.8689630966648629

In [49]:
score

0.8689630966648629

In [50]:
print("score", score)
oofs_dir = "../data/ensemble/second_level/oofs/"
output_dir = "../data/ensemble/second_level/preds/"
onehot_pred_dir = "../data/ensemble/second_level/nn_one_hot/"

model_submit_prefix = "AddNN-Ensemble"

oofs_path = oofs_dir + model_submit_prefix
output_path = output_dir + model_submit_prefix
one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

print("Predicting training results...")
train_predicts = np.concatenate(folds_preds, axis=0)
score = np_weighted_accuracy(to_categorical(labels), train_predicts)

oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
oofs.to_csv(submit_path, index=False)

print("Predicting testing results...")
test_predicts_list = []
for fold_id, model in enumerate(models):
    test_predicts = model.predict({"mata-features": ensemble_tests}, batch_size=128, verbose=1)
    test_predicts_list.append(test_predicts)

test_predicts = np.zeros(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts += fold_predict
test_predicts /= len(test_predicts_list)

test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
test_predicts.to_csv(submit_path, index=False) # 0.3343

print("Predicting labeled testing results...")
ids = pd.read_csv("../data/dataset/test.csv")
pred_labels = test_predicts.idxmax(axis=1)
sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
sub.to_csv(submit_path, index=False)

score 0.8689630966648629
Predicting training results...
Predicting testing results...
80126/80126 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 9us/step
Predicting labeled testing results...
